#Code for the Service using Web.py
This contains all necessary code block to create the service. Main code file contains:
- app.py (for initiating the web service)
- controller.py (for directing the request and respond)
- caffe_model.py (for applying the various caffe model to generate a respond)

##app.py

Adapted code from: https://gist.github.com/w0rm/3907294

In [5]:
%%writefile app.py
import web
import json
from controller import Controller
from caffe_models import Caffe_Models

urls = (
    r'/resources(?:/(?P<resource_id>[0-9]+))?',
    'ResourceController',
)

class ResourceController(Controller):

    def list(self):
        return "list resources", format
    
    #MODELS THAT WE USE    
    def style_flickr(self):
        request = json.loads(web.data())
        cm = Caffe_Models('style_flickr')
        return cm.caffe_models_run(request) #response

    def object_lenet(self):
        request = json.loads(web.data())
        cm = Caffe_Models('object_lenet')
        return cm.caffe_models_run(request) #response
    
    def place_lenet(self):
        request = json.loads(web.data())
        cm = Caffe_Models('place_lenet')
        return cm.caffe_models_run(request) #response
    
    
    #MODELS THAT WE ARE NOT USING

    ## this is the baseline
    def object_imagenet(self):
        request = json.loads(web.data())
        cm = Caffe_Models('object_imagenet')
        return cm.caffe_models_run(request) #response

    def place_cnn(self):
        request = json.loads(web.data())
        cm = Caffe_Models('place_cnn')
        return cm.caffe_models_run(request) #response

    
    
    
## ORIGINAL RESTFUL CONTROLLER
#     def get(self, resource_id):
#         return "retrieved resource", resource_id

#     def create(self):
#         resource = json.loads(web.data())
#         return "created resource", resource

#     def update(self, resource_id):
#         resource = json.loads(web.data())
#         return "updated resource", resource_id, resource

#     def delete(self, resource_id):
#         return "deleted resource", resource_id

app = web.application(urls, globals())

if __name__ == "__main__":
    app.run()

Overwriting app.py


##controller.py

In [4]:
%%writefile controller.py
import web

class Controller:

    methods = ("list", "get", "create", "update", "delete",
               "update_collection", "delete_collection")

    def __getattr__(self, name):
        if name in self.methods and "headers" in web.ctx:
            raise web.badrequest()
        else:
            raise AttributeError

    def POST(self, resource_id=None):
        if resource_id=="1":
            return self.style_flickr()
        elif resource_id=="2":
            return self.object_lenet()
        elif resource_id=="3":
            return self.place_lenet()
        else:
            raise web.badrequest()

## ORIGINAL CRUD OPERATION FROM RESTFUL CONTROLLER            
#     def GET(self, resource_id=None):
#         if resource_id is None:
#             return self.list()
#         else:
#             return self.get(resource_id)
#     #READ - TEST in browswer: http://localhost:8080/resources/23
        
#     def POST(self, resource_id=None):
#         if resource_id is None:
#             return self.create()
#         else:
#             raise web.badrequest()
#     #CREATE - TEST in terminal: curl -X POST  -H "Content-Type: application/json" -d '{"title":"a title","description":"a description","type":"a type","author":"an author"}' -i http://localhost:8080/resources/
    
#     def PUT(self, resource_id=None):
#         if resource_id is None:
#             return self.update_collection()
#         else:
#             return self.update(resource_id)
#     #UPDATE: TEST in terminal: curl -X PUT  -H "Content-Type: application/json" -d '{"id":"23", "title":"a title","description":"a description","type":"a type","author":"an author"}' -i http://localhost:8080/resources/23
    
#     def DELETE(self, resource_id=None):
#         if resource_id is None:
#             return self.delete_collection()
#         else:
#             return self.delete(resource_id)
#     #DELETE: TEST in terminal: curl -X DELETE http://localhost:8080/resources/23

Overwriting controller.py


In [12]:
%%writefile caffe_models.py
#Grab new images
import ast
import os
import json
import urllib
import posixpath
import urlparse 
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os
import caffe
import sys

class Caffe_Models():
    
    def __init__(self, model_name):
        if model_name=='style_flickr':
            self.model_path = 'models/finetune_flickr_style/finetune_flickr_style.caffemodel'
            self.model_labels = 'examples/finetune_flickr_style/style_names.txt'
            self.model_deploy='models/finetune_flickr_style/deploy.prototxt'
            self.model_mean='python/caffe/imagenet/ilsvrc_2012_mean.npy'
            self.x_size=227
            self.y_size=227
        elif model_name=='place_lenet':
            self.model_path = 'models/googlenet_places205/googlelet_places205_train_iter_2400000.caffemodel'
            self.model_labels = 'models/placesCNN/categoryIndex_places205.csv'
            self.model_deploy='models/googlenet_places205/deploy_places205.protxt'
            self.model_mean='python/caffe/imagenet/ilsvrc_2012_mean.npy'
            self.x_size=224
            self.y_size=224
        elif model_name=='object_lenet': 
            self.model_path = 'models/bvlc_googlenet/bvlc_googlenet.caffemodel'
            self.model_labels = 'data/ilsvrc12/synset_words.txt'
            self.model_deploy='models/bvlc_googlenet/deploy.prototxt'
            self.model_mean='python/caffe/imagenet/ilsvrc_2012_mean.npy'
            self.x_size=224
            self.y_size=224
#         elif model_name=='object_imagenet':
#             self.model_path ='models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'
#             self.model_labels = 'data/ilsvrc12/synset_words.txt'
#             self.model_deploy='models/bvlc_reference_caffenet/deploy.prototxt'
#         elif model_name=='place_cnn':
#             self.model_path = 'models/placesCNN/places205CNN_iter_300000.caffemodel'
#             self.model_labels = 'models/placesCNN/categoryIndex_places205.csv'
#             self.model_deploy='models/placesCNN/places205CNN_deploy.prototxt'
              
        self.index=0
        self.limit=999
        self.num_images=0
        self.num_guesses=5
        self.images = {}
        self.predictions = {}
        self.DATA_PATH = "../../../dataset/"
        self.caffe_root = '../../../caffe/'

    #main function for running the baseline script
    def caffe_models_run(self,request):
        self.generate_image(request)
        self.caffe_setup()
        self.caffe_predict()
        return self.predictions

    #generate images
    def generate_image(self,request):
        #clean up temp file
        for f in os.listdir(self.DATA_PATH+"temp/"):
            os.remove(self.DATA_PATH+"temp/"+f)
           
        #for each feed_id, create image and set index for images
        for feed_id, image_url in request.iteritems():
            
            #read json but need to handle index error as certain lines of JSON has errors
            #during as we attempt to extract the value in image_url
            try:
                #figure out filename
                path = urlparse.urlsplit(image_url).path
                filename = posixpath.basename(path)
                image_path = self.DATA_PATH+"temp/"+filename
                #download to local folder and resize image
                urllib.urlretrieve(image_url,image_path)
                
                #need to handle potential error due to issues not being able to open image file
                try: 
                    img = Image.open(image_path)
                    img.thumbnail((256,256),Image.ANTIALIAS) #best quality
                    #set limit on number of images to process
                    self.index+=1
                    print "file no.",self.index,"processed:",image_path
                    self.images[feed_id]=image_path
                    self.num_images+=1
                    if self.index==self.limit: break
                except IOError:
                    print "Cannot open image: remove image and go to next line of JSON"
                    os.remove(image_path)
                    self.images[feed_id]="NA"

            except IndexError:
                print "Index error with line of JSON: ignore and go to next line of JSON"
                self.images[feed_id]="NA"

                
                
    #Import required modules, set plotting parameters, and run 
    #./scripts/download_model_binary.py models/bvlc_reference_caffenet 
    #to get the pretrained CaffeNet model if it hasn't yet been fetched
    def caffe_setup(self):
        # Make sure that caffe is on the python path:
        sys.path.insert(0, self.caffe_root + 'python')

        plt.rcParams['figure.figsize'] = (10, 10)
        plt.rcParams['image.interpolation'] = 'nearest'
        plt.rcParams['image.cmap'] = 'gray'

#         ##No need to download as Roselind already downloaded the file
#         if not os.path.isfile(self.caffe_root + self.model_path):
# #         if not os.path.isfile(self.caffe_root + 'models/finetune_flickr_style/finetune_flickr_style.caffemodel'):
# #         if not os.path.isfile(self.caffe_root + 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'):
#             print("Downloading pre-trained "+self.model_long_name+" model...")   
# #             print("Downloading pre-trained finetune_flickr_style model...")   
# #             print("Downloading pre-trained CaffeNet model...")       
#             #QUESTIONS... DIDN"T CHANGE!!!
#             os.system(self.caffe_root+"scripts/download_model_binary.py caffe/models/bvlc_reference_caffenet")
        
    #Make image predictions
    def caffe_predict(self):
        
        # Set Caffe to CPU mode, load the net in the test phase for inference, and configure input preprocessing.
        caffe.set_mode_cpu()
        ##CHANGE
        net = caffe.Net(self.caffe_root + self.model_deploy,
                        self.caffe_root + self.model_path,
                        caffe.TEST)
#         net = caffe.Net(self.caffe_root + 'models/finetune_flickr_style/deploy.prototxt',
#                         self.caffe_root + 'models/finetune_flickr_style/finetune_flickr_style.caffemodel',
#                         caffe.TEST)
#         net = caffe.Net(self.caffe_root + 'models/bvlc_reference_caffenet/deploy.prototxt',
#                     self.caffe_root + 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel',
#                     caffe.TEST)


        # input preprocessing: 'data' is the name of the input blob == net.inputs[0]
        transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
        transformer.set_transpose('data', (2,0,1))
        ## QUESTION
        transformer.set_mean('data', np.load(self.caffe_root + self.model_mean).mean(1).mean(1)) # mean pixel
        transformer.set_raw_scale('data', 255)  # the reference model operates on images in [0,255] range instead of [0,1]
        transformer.set_channel_swap('data', (2,1,0))  # the reference model has channels in BGR order instead of RGB

        # set net to batch size to be the total number of images
        net.blobs['data'].reshape(self.num_images,3,self.x_size,self.y_size)

        
        # load labels
        ##CHANGE
        labels_filename = self.caffe_root + self.model_labels
#         imagenet_labels_filename = self.caffe_root + 'examples/finetune_flickr_style/style_names.txt'
#         imagenet_labels_filename = self.caffe_root + 'data/ilsvrc12/synset_words.txt'
        try:
        ##CHANGE
            labels = np.loadtxt(labels_filename, str, delimiter='\t')
#             labels = np.loadtxt(imagenet_labels_filename, str, delimiter='\t')
        except:
#             !../data/ilsvrc12/get_ilsvrc_aux.sh
            os.system(self.caffe_root + 'data/ilsvrc12/get_ilsvrc_aux.sh')

        ##CHANGE
            labels = np.loadtxt(labels_filename, str, delimiter='\t')
#             labels = np.loadtxt(imagenet_labels_filename, str, delimiter='\t')

        # load image data into model
#         for (dirpath, dirnames, filenames) in walk(self.DATA_PATH+"temp/"):
        i=0
        image_ids = {}
        for feed_id, img_path in self.images.iteritems():
            #load image into caffe preprocessor
            net.blobs['data'].data[i] = transformer.preprocess('data',  
                caffe.io.load_image(img_path))
            image_ids[feed_id]=i
            i+=1

        #batch computation step (show computation time)
        net.forward()  # call once for allocation
#         %timeit net.forward()
        
        #display result, showing top 5 prediction for each image
#         for (dirpath, dirnames, filenames) in walk('../../../dataset/images/'):
#             for i,filename in enumerate(filenames):
        for feed_id,i in image_ids.iteritems():
            probs = net.blobs['prob'].data[i].flatten()
            top_probs=probs[-1:-1-self.num_guesses:-1]
            top_k = probs.argsort()[-1:-1-self.num_guesses:-1]
        
            top_predictions = labels[top_k]

            for top_prob,top_prediction in zip(top_probs,top_predictions):
                self.predictions.setdefault(feed_id, [])
                pred_id = top_prediction[:9]
                pred_name = top_prediction[11:]
                self.predictions[feed_id].append({"id":pred_id,"name":pred_name,"score":top_prob}) 


Overwriting caffe_models.py


In [15]:
#test code
!chmod +x app.py
!chmod +x controller.py
!chmod +x caffe_models.py

%reload_ext autoreload
%autoreload 2

from caffe_models import Caffe_Models

request = {"1094301226639611969_1016687101": "https://scontent.cdninstagram.com/hphotos-xfa1/t51.2885-15/e35/12071132_117024925320109_1448778029_n.jpg", "1079932954249514250_1009689594": "https://scontent.cdninstagram.com/hphotos-xaf1/t51.2885-15/s640x640/sh0.08/e35/11377801_146811182334693_2053033951_n.jpg", "901390003869460339_1016673720": "https://scontent.cdninstagram.com/hphotos-xft1/t51.2885-15/e15/10919675_339630669566407_1560505453_n.jpg", "1091411570335070017_1002181196": "https://scontent.cdninstagram.com/hphotos-xaf1/t51.2885-15/s640x640/sh0.08/e35/12139791_172476163092798_682238343_n.jpg", "1085670910110935094_1001957249": "https://scontent.cdninstagram.com/hphotos-xfa1/t51.2885-15/e35/11899529_409336062593154_26859144_n.jpg"}
# b = Baseline()
# respond = b.baseline_run(request)
# print respond

cm = Caffe_Models('style_flickr')
# cm = Caffe_Models('place_lenet')
# cm = Caffe_Models('object_lenet')
respond = cm.caffe_models_run(request) #response
print respond

file no. 1 processed: ../../../dataset/temp/12071132_117024925320109_1448778029_n.jpg
file no. 2 processed: ../../../dataset/temp/11899529_409336062593154_26859144_n.jpg
file no. 3 processed: ../../../dataset/temp/12139791_172476163092798_682238343_n.jpg
file no. 4 processed: ../../../dataset/temp/10919675_339630669566407_1560505453_n.jpg
file no. 5 processed: ../../../dataset/temp/11377801_146811182334693_2053033951_n.jpg
{'1094301226639611969_1016687101': [{'score': 0.093098111, 'id': 'Pastel', 'name': ''}, {'score': 0.00094565586, 'id': 'Vintage', 'name': ''}, {'score': 0.009112522, 'id': 'Romantic', 'name': ''}, {'score': 0.00051206991, 'id': 'Melanchol', 'name': ''}, {'score': 0.0016330597, 'id': 'Bright', 'name': ''}], '901390003869460339_1016673720': [{'score': 0.065550283, 'id': 'Vintage', 'name': ''}, {'score': 0.00089605176, 'id': 'Bokeh', 'name': ''}, {'score': 0.00041621225, 'id': 'Pastel', 'name': ''}, {'score': 0.083875611, 'id': 'Depth of ', 'name': 'eld'}, {'score': 4.8

#Appendix: For Testing Web.py

In [15]:
%%writefile service.py
import web
import json
# import index

urls = (
    '/', 'index'
)

class index:
    #for testing purpose
    def GET(self):
        pyDict = {'test1':'works','test2':"works too"}
        web.header('Content-Type', 'application/json')
        return json.dumps(pyDict)

    def POST(self):
#         pyDict = {'one':1,'two':2}
        pyDict = web.input()
        web.header('Content-Type', 'application/json')
        return json.dumps(pyDict)

##ORIGINAL TEST CODE
# class index:
#     def GET(self):
#         return "Hello1, world!"
    
    
if __name__ == "__main__":
    app = web.application(urls, globals())
    app.run()

Overwriting service.py


In [7]:
!chmod +x service.py
!python service.py

http://0.0.0.0:8080/
^C
